Run the following command in the terminal to install postgres and psql on this Cloud9 instance.

In [ ]:
# sudo amazon-linux-extras enable postgresql12
# sudo yum clean metadata
# sudo yum install postgresql
# psql --version


Now, run the below command to set the required environment variables.

In [ ]:
# ROLE_ARN=$(aws cloudformation describe-stacks \
#   --stack-name 'using-genai-for-private-files-workshop' \
#   --query 'Stacks[0].Outputs[?OutputKey==`AmazonBedrockExecutionRoleARN`].OutputValue' \
#   --output text)
# echo $ROLE_ARN
# REGION=$(aws configure get region)
# echo $REGION
# Aurora_Cluster_Writer_Endpoint=$(aws cloudformation describe-stacks \
#   --stack-name 'using-genai-for-private-files-workshop' \
#   --query 'Stacks[0].Outputs[?OutputKey==`AuroraDBWriterEndpoint`].OutputValue' \
#   --output text)
# echo $Aurora_Cluster_Writer_Endpoint
# Aurora_Cluster_Arn=$(aws cloudformation describe-stacks \
#   --stack-name 'using-genai-for-private-files-workshop' \
#   --query 'Stacks[0].Outputs[?OutputKey==`AuroraDBClusterARN`].OutputValue' \
#   --output text)
# echo $Aurora_Cluster_Arn
# Postgres_Bucket_Name=$(aws cloudformation describe-stacks \
#   --stack-name 'using-genai-for-private-files-workshop' \
#   --query 'Stacks[0].Outputs[?OutputKey==`WorkshopPostgreS3BucketName`].OutputValue' \
#   --output text)
# echo $Postgres_Bucket_Name
# Postgres_Bucket_Arn=$(aws cloudformation describe-stacks \
#   --stack-name 'using-genai-for-private-files-workshop' \
#   --query 'Stacks[0].Outputs[?OutputKey==`WorkshopPostgreS3BucketARN`].OutputValue' \
#   --output text)
# echo $Postgres_Bucket_Arn
# DB_Password=$(aws secretsmanager get-secret-value --secret-id postgres_vectors_secrets --query SecretString --output text | jq -r .password)
# echo "DB_Password: $DB_Password"


Use the following command in the terminal to connect to the DB cluster and use the use the password printed in the terminal above.

<!-- psql -h $Aurora_Cluster_Writer_Endpoint -d postgres -U workshopUser -p 3306 -->


To create an extension named "vector" in a PostgreSQL database, run the following command in Cloud9 terminal.

<!-- CREATE EXTENSION IF NOT EXISTS vector; -->


Use the following command to retrieve the version number of the installed 'vector' extension in a PostgreSQL database.

In [ ]:
# SELECT extversion FROM pg_extension WHERE extname='vector';


<!-- SELECT extversion FROM pg_extension WHERE extname='vector'; -->


Run the following command in Cloud9 terminal to create a new schema named bedrock_integration in a PostgreSQL database.

In [ ]:
# CREATE SCHEMA bedrock_integration;


<!-- CREATE SCHEMA bedrock_integration; -->


In the Cloud9 terminal, enter the following command to create a new user and grant required permissions

<!-- CREATE ROLE bedrock_user WITH PASSWORD 'workshoppassword@1' LOGIN;
GRANT ALL ON SCHEMA bedrock_integration to bedrock_user; -->


Then log in using the new user and password you just created.

<!-- \q 
psql -h $Aurora_Cluster_Writer_Endpoint -d postgres -U bedrock_user -p 3306 -->


Now, create a table with the right schema using the below command in the Cloud9 terminal.

<!-- CREATE TABLE bedrock_integration.bedrock_kb (id uuid PRIMARY KEY, embedding vector(1536), chunks text, metadata json); -->


Create an index named bedrock_kb on the embedding column of a table within the bedrock_integration schema, using the vector_cosine_ops operator class in PostgreSQL

<!-- CREATE INDEX on bedrock_integration.bedrock_kb USING hnsw (embedding vector_cosine_ops);
\q -->


Add the secrets to secrets manager
Go to the AWS Secrets Manager Console 
Click the button Store a new secret
Select secret type as Credentials for Amazon RDS database
Provide the username bedrock_user and password workshoppassword@1
Under Database, select workshopdbcluster
Click Next and provide secret name postgres_kb_secret
Click Next > Next > Store

To retrieve the ARN of the secrets you created above, use the following command in your Cloud9 terminal.

<!-- SECRET_ARN=$(aws secretsmanager list-secrets --query 'SecretList[?Name==`postgres_kb_secret`].ARN' --output text)
echo $SECRET_ARN -->


To create knowledge base in Amazon Bedrock, use the following command in Cloud9 terminal.

<!-- Knowledge_Base_Id=$(aws bedrock-agent create-knowledge-base \
  --name workshop-aurora-knowledge-base \
  --role-arn $ROLE_ARN \
  --knowledge-base-configuration 'type=VECTOR,vectorKnowledgeBaseConfiguration={embeddingModelArn="arn:aws:bedrock:'"$REGION"'::foundation-model/amazon.titan-embed-text-v1"}' \
  --storage-configuration 'type=RDS,rdsConfiguration={resourceArn='"$Aurora_Cluster_Arn"',credentialsSecretArn='"$SECRET_ARN"',databaseName=postgres,tableName=bedrock_integration.bedrock_kb,fieldMapping={primaryKeyField=id,vectorField=embedding,textField=chunks,metadataField=metadata}}'\
  | jq -r '.knowledgeBase.knowledgeBaseId')
  echo $Knowledge_Base_Id -->


Follow the steps below to add a secret value in secrets manager

Go to the AWS Secrets Manager Console 
Click on postgres_vectors_secrets secret
Under Secret value click Retrieve secret value
Then, click Edit
Click + Add row
For key type KNOWLEDGE_BASE_ID and for the value paste the value of KNOWLEDGE_BASE_ID printed in the terminal above
Click Save

To create a data store within the knowledge base, use the following command in Cloud9 terminal.



<!-- Data_Source_Id=$(aws bedrock-agent create-data-source \
  --knowledge-base-id $Knowledge_Base_Id \
  --name workshop-aurora-kb-data-source \
  --data-source-configuration "type=S3,s3Configuration={bucketArn='"$Postgres_Bucket_Arn"'}" \
  --vector-ingestion-configuration "chunkingConfiguration={chunkingStrategy=NONE}" \
  | jq -r '.dataSource.dataSourceId')
echo $Data_Source_Id -->


In the Cloud9 terminal, run the following command to download the dataset using LangChain wikipedia retriever.

<!-- python download_dataset.py -->


In the Cloud9 terminal, run the following command to upload a dataset to Amazon S3.

<!-- aws s3 cp ./docs s3://${Postgres_Bucket_Name}/ --recursive -->


In the Cloud9 terminal, run the following command to sync data in knowledge-base

<!-- aws bedrock-agent start-ingestion-job --knowledge-base-id $Knowledge_Base_Id --data-source-id $Data_Source_Id -->


In the Cloud9 terminal, enter the following command to run the Streamlit app.


<!-- streamlit run app.py -->


The output will display two IP addressess. Open the endpoint with the External IP address.